# Figure out of a uvdata object is bl-slicable

In [1]:
import numpy as np
from pyuvdata import UVData, UVCal
from hera_cal import io
from pyuvdata.utils import polnum2str, polstr2num, jnum2str, jstr2num
from hera_cal.datacontainer import DataContainer

In [9]:
%load_ext snakeviz

In [10]:
uvd = UVData()

In [11]:
%load_ext line_profiler

In [23]:
from pyuvdata.miriad import Miriad

In [24]:
%lprun -f Miriad.read_miriad uvd.read_miriad('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/')

In [17]:
#%snakeviz uvd.read_miriad('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/')
%snakeviz io.load_vis(uvd)

 
*** Profile stats marshalled to file u'/var/folders/2s/n8w45_q92jgcl5bm1pdgsqr40000gr/T/tmphf6_7H'. 


# HERAData

In [13]:
uvd = UVData()
# uvd.read_miriad('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv')
# uvd.write_uvh5('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.h5')

In [15]:
#uvd.write_uvh5('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.h5', clobber=True)

In [16]:
uvd.read_uvh5('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.h5')

In [72]:
import operator
import collections
from collections import OrderedDict as odict
from copy import deepcopy

#todo: in legacy support in load_vis, if it's a uvdata object, set it's .__class__ to HERAData


def split_pol(pol):
    '''Splits visibility polarization string into anntenna polarizations.
    Can be updated if and when we move to Jones-matrix formalism.'''
    return pol[0], pol[1]

def conj_pol(pol):
    '''Given V_ij^(pol), return the polarization of V_ji^(conj pol) such 
    (V_ji^(conj pol))* = V_ij^(pol). This means xy -> yx and yx -> xy, but
    psuedo-Stokes visibilities are unaffected.'''
    if polstr2num(pol) > 0:  # this includes Stokes and pseudo-Stokes visibilities 
        return pol
    else:
        return pol[::-1]


class HERACal(UVCal):
    
    def __init__(self, input_cal):
        '''TODO: document'''
        super(HERACal, self).__init__()
        
        # parse input_data as filepath(s)
        if isinstance(input_cal, str):
            self.filepaths = [input_cal]
        elif isinstance(input_cal, collections.Iterable):  # List loading
            if np.all([isinstance(i, str) for i in input_cal]):  # List of visibility data paths
                self.filepaths = list(input_data)
            else:
                raise TypeError('If input_cal is a list, it must be a list of strings.')
        else: 
            raise ValueError('input_cal must be a string or a list of strings.')
            
        # TODO: when calfits partial / metadata loading is enabled, load metadata here
    
    def _extract_metadata(self):
        '''TODO: document'''
        ants = self.ant_array
        freqs = np.unique(self.freq_array)
        times = np.unique(self.time_array)
        pols = [jnum2str[j] for j in self.jones_array]
      
    def _build_cal_dicts(self, key):
        '''TODO: document'''
        gains, quals, flags, self.total_qual = odict(), odict(), odict(), odict()
        for ip, pol in enumerate(self.jones_array):
            if self.total_quality_array is not None:
                self.total_qual[jonesnum2str[pol]] = self.total_quality_array[0, :, :, ip].T
            else:
                self.total_qual = None
            for i, ant in enumerate(cal.ant_array):
                gains[(ant, jnum2str[pol])] = np.array(self.gain_array[i, 0, :, :, ip].T)
                flags[(ant, jnum2str[pol])] = np.array(self.flag_array[i, 0, :, :, ip].T)
                quals[(ant, jnum2str[pol])] = np.array(self.quality_array[i, 0, :, :, ip].T)
                
        return gains, flags, quals

    def read_cal(self):
        '''TODO: document'''
        try:
            return self._build_cal_dicts()
        except:
            self.read_calfits(self.filepaths)
            self._extract_metadata()
            return self._build_cal_dicts()        
        
    
class HERAData(UVData):
    
    
    '''MY IDEA: MAKE A HERA DATA OBJECT INSTANTIATED WITH A FILE NAME THAT LOADS IN META DATA AND PUTS IT IN A NICE FORM
    THIS THEN PROVIDES AN INTERFACE TO PARTIAL DATA LOADING AND ALSO PARTIAL DATA WRITING WHICH SPITS OUT DATA CONTAINERS WHICH HAVE COPIES
    OF THE DATA. '''

    def __init__(self, input_data, filetype='uvh5'):
        '''TODO: document'''
        # initialize as empty UVData object
        super(HERAData, self).__init__()
        self.filetype = filetype
        
        # parse input_data as filepath(s)
        if isinstance(input_data, str):
            self.filepaths = [input_data]
        elif isinstance(input_data, collections.Iterable):  # List loading
            if np.all([isinstance(i, str) for i in input_data]):  # List of visibility data paths
                self.filepaths = list(input_data)
            else:
                raise TypeError('If input_data is a list, it must be a list of strings.')
        else: 
            raise ValueError('input_data must be a string or a list of strings.')
        
        # load metadata from file
        elif self.filetype is 'miriad':
            self.read_miriad_metadata(self.filepaths[0])
        elif self.filetype is 'uvfits':
            self.read_uvfits_metadata(self.filepaths[0])
        else:
            raise NotImplementedError('Filetype ' + self.filetype + ' has not been implemented.')
        self.previous_read = None
           
        # extract array information
        antpos, ants = self.get_ENU_antpos()
        self.ants = sorted(ants)
        self.antpos = dict(zip(self.ants, antpos))
        
        # extra calibration and flagging data
        self._flags = None
        self._gains = None
        self._cal_flags = None
    
  
    def apply_vis_flags(self, vis_flags_dict):
        '''TODO: document'''
        self._flags = {(ant1, ant2, polstr2num(pol)): vis_flags_dict[ant1, ant2, pol] 
                                                      for (ant1, ant2, pol) in vis_flags_dict.keys()}

    def apply_calibration(self, gain_dict):
        '''TODO: document'''
        self._gains = {(ant, jstr2num(ap)): gain_dict[ant, ap] for (ant, ap) in gain_dict.keys()}

    def apply_cal_flags(self, cal_flags_dict):
        '''TODO: document'''
        self._cal_flags = {(ant, jstr2num(ap)): gain_dict[ant, ap] for (ant, ap) in gain_dict.keys()}
        
    def apply_HERACal(self, hc):
        '''TODO: document'''
        gains, flags, quals = hc.read_cal()
        self.apply_calibration(gains)
        self.apply_cal_flags(flags)
    
    def _extract_metadata(self):
        '''TODO: document'''
        self.freqs = np.unique(self.freq_array)
        self.times = np.unique(self.time_array)
        lst_indices = np.unique(self.lst_array.ravel(), return_index=True)[1]
        self.lsts = self.lst_array.ravel()[np.sort(lst_indices)]
        self.pols = np.array([polnum2str(polnum) for polnum in self.polarization_array])
        self.antpairs = self.get_antpairs()
        self.bls = [antpair + (pol,) for antpair in self.antpairs for pol in self.pols]
        
        self._determine_blt_slicing()
        self._determine_pol_indexing()
        self.times_by_antpair = {antpair: np.array(self.time_array[self._blt_slices[antpair]]) 
                                          for antpair in self.antpairs}
        self.lsts_by_antpair = {antpair: np.array(self.lst_array[self._blt_slices[antpair]]) 
                                         for antpair in self.antpairs}        

    def _determine_blt_slicing(self):
        '''TODO: document'''
        self._blt_slices = {}
        for ant1, ant2 in self.antpairs:
            indices = self.antpair2ind(ant1, ant2)
            if not (len(set(np.ediff1d(indices))) == 1):
                raise NotImplementedError('UVData objects with non-regular spacing of baselines in its baseline-times are not supported.')
            self._blt_slices[(ant1, ant2)] = slice(indices[0], indices[-1] +1 , indices[1] - indices[0])
         
    def _determine_pol_indexing(self):
        '''TODO: document'''
        self._polnum_indices = {}
        for i, polnum in enumerate(self.polarization_array):
            self._polnum_indices[polnum] = i

    def _get_slice(self, data_array, key):
        '''TODO: REWRITE... Returns the data corresponding to the key. If the key is just a polarization,
        returns all baselines for that polarization. If the key is just a baseline,
        returns all polarizations for that baseline. If the key is of the form (0,1,'xx'),
        returns the associated entry. Abstracts away both baseline ordering (applying the
        complex conjugate when appropriate) and polarization capitalization.  NB: makes copies of data'''
        if isinstance(key, tuple) and len(key) == 3:  # asking for bl-pol
            try:
                out = np.conj(np.squeeze(data_array[self._blt_slices[key[1::-1]], 0, :, 
                              self._polnum_indices[polstr2num(conj_pol(key[2]))]]))
            except:
                out = np.array(np.squeeze(self.data_array[self._blt_slices[key[0:2]], 0, :, 
                                 self._polnum_indices[polstr2num(key[2])]]))
                
            if self._gains is not None:  # divide out calibration solutions
                jn_i, jn_j = jstr2num([jstr for jstr in split_pol(key[2])])
                out = out / self._gains[key[0], jn_i] / np.conj(self._gains[key[1], jn_j])
            
            if self._cal_flags is not None:  # incorporate per-antenna flags
                jn_i, jn_j = split_pol(key[2])
                out = out | self._cal_flags[key[0], jstr2num(jn_i)] | self._cal_flags[key[1], jstr2num(jn_j)]

            if self._flags is not None:  # incorporate visibility flags
                try:
                    out = out | self._flags[(key[0], key[1], polstr2num(key[2]))]
                except:
                    out = out | self._flags[key[1], key[0], polstr2num(conj_pol(key[2]))]
               
            return out
        
        elif isinstance(key, tuple) and len(key) == 2:  # asking for antpair
            return {pol: self._get_slice(key + (pol,)) for pol in self.pols}
        elif isinstance(key, str):  # asking for a pol
            return {antpair: self._get_slice(antpair + (key,)) for antpair in self.antpairs}
        else:
            raise KeyError('Unrecognized key type for slicing data.')
        
    
    def _build_datacontainers(self):
        '''TODO: document'''
        data, flags, nsamples = odict(), odict(), odict()
        for bl in self.bls:
            data[bl] = self._get_slice(self.data_array, bl)
            flags[bl] = self._get_slice(self.flag_array, bl)
            nsamples[bl] = self._get_slice(self.nsample_array, bl)

        data = DataContainer(data)
        flags = DataContainer(flags)
        nsamples = DataContainer(nsamples)
        
        # store useful metadata inside the DataContainers
        for dc in [data, flags, nsamples]:
            dc.freqs = self.freqs
            dc.times = self.times
            dc.lsts = self.lsts
            dc.times_by_antpair = self.times_by_antpair
            dc.lsts_by_antpair = self.lsts_by_antpair
            
        return data, flags, nsamples
     
        
    def avoid_double_read(read_data):
        '''Decorator that avoids the need to re-reading data if the request matches the previous one.'''
        def wrapper(self, **kwargs):
            if self.previous_read == kwargs:
                return self._build_datacontainers()
            else:
                self.previous_read = kwargs
                return read_data(self, **kwargs)
        return wrapper
    
    
    @avoid_double_read
    def read_data(self, bls=None, antenna_nums=None, polarizations=None, ant_str=None):
        '''
        
        
        TODO: implement partial loading by frequency and/or channel and time and/or integration
        
        Arguments:
            bls: A list of antenna number tuples (e.g. [(0,1), (3,2)]) or a list of
                baseline 3-tuples (e.g. [(0,1,'xx'), (2,3,'yy')]) specifying baselines
                to keep in the object. For length-2 tuples, the  ordering of the numbers
                within the tuple does not matter. For length-3 tuples, the polarization
                string is in the order of the two antennas. If length-3 tuples are
                provided, the polarizations argument below must be None.
            antenna_nums: The antennas numbers to read into the object.
            polarizations: List of polarization integers or strings to read-in.
                Ex: ['xx', 'yy', ...]
            ant_str: A string containing information about what kinds of visibility data
                to read-in.  Can be 'auto', 'cross', 'all'. Cannot provide ant_str if
                antenna_nums and/or bls is not None.
        
        Returns:
            data: 
            flags: 
            nsamples: 
        '''
        
#         try:
#             return self._build_datacontainers()
#         except:
        if self.filetype is 'miriad':
            print 'reading miriad'
            self.read_miriad(self.filepaths, bls=bls, antenna_nums=antenna_nums, 
                             polarizations=polarizations, ant_str=ant_str)
        elif self.filetype is 'uvfits':
            self.read_uvfits(self.filepaths, bls=bls, antenna_nums=antenna_nums, 
                             polarizations=polarizations, ant_str=ant_str)
            self.unphase_to_drift()
        self._extract_metadata() 
        return self._build_datacontainers()

    
    def _set_slice(self, data_array, key, value):
        '''TODO: REWRITE... Sets the data corresponding to the key. 
        Abstracts away both baseline ordering (applying the complex conjugate when
        appropriate) and polarization capitalization.'''
        if isinstance(key, tuple) and len(key) == 3:  # providing bl-pol
            try:
                self._data_array[self._blt_slices[key[0:2]], 0, : , 
                                 self._polnum_indices[polstr2num(key[2])]] = value
            except(KeyError):
                self._data_array[self._blt_slices[key[1::-1]], 0, : , 
                                 self._polnum_indices[polstr2num(conj_pol(key[2]))]] = np.conj(value)
        elif isinstance(key, tuple) and len(key) == 2:  # providing antpair with all pols
            for pol in value.keys():
                self._set_slice(data_array, (key + (pol,)), value[pol])
        elif isinstance(key, str):  # providing pol with all antpairs
            for antpair in value.keys():
                self._set_slice(data_array, (antpair + (key,)), value[key])
        else:
            raise KeyError('Unrecognized key type for slicing data.')
    
    def set_data(self, data=None, flags=None, nsamples=None):
        '''TODO: document'''
        
        if data is not None:
            for bl in data.keys():
                self._set_slice(self.data_array, bl, data[bl])
        if flags is not None:
            for bl in flags.keys():
                self._set_slice(self.flag_array, bl, flags[bl])
        if data is not None:
            for bl in nsamples.keys():
                self._set_slice(self.nsample_array, bl, nsamples[bl])
        
        
    def __getitem__(self, key):
        '''TODO: document'''
        return self.read_data(bls=key)[0][key]
    
    def iterate_over_antpairs(self, nbl=1):
        '''TODO: document'''
        ants = deepcopy(self.ants)
        for i, ant1 in enumerate(ants):
            for ant2 in ants[i:]:
                #TODO: this is not working!!!!!!!!!
                data, flags, nsamples = self.read_data(bls=(ant1, ant2))
                yield data, flags, nsamples
                
        
        
    
#    def iter_read_data or read_data_by_bl / read_data_by_time / read_data_by_chan

In [73]:
hd = HERAData('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/')

In [ ]:
hd.read_uvh5()

In [76]:
d,f,n = hd.read_data(bls=[(1,2,'XX')], polarizations=None)
hd.get_antpairs()
print d.keys()

[(1, 2, 'XX')]


In [65]:
uvd = UVData()
uvd.read_miriad_metadata('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/')

In [66]:
uvd.nsample_array

In [67]:
uvd = UVData()
uvd.read_miriad('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/', bls=(1,2))
print uvd.get_antpairs()
uvd.read_miriad('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/', bls=(0,1))
print uvd.get_antpairs()

[(1, 2)]
[(0, 1)]


In [68]:
uvd.get_antpairs()

[(0, 1)]

In [12]:
hd._blt_slices

{(1, 2): slice(0, 60, 1)}

In [13]:
for data, flags, nsamples in hd.iterate_over_antpairs():
    print data.keys()

reading miriad
ap1: [(0, 0)]
ap2: [(0, 0)] [(0, 0, 'XX')]
[(0, 0, 'XX')]
reading miriad
ap1: [(0, 1)]
ap2: [(0, 1)] [(0, 1, 'XX')]
[(0, 1, 'XX')]
reading miriad
ap1: [(0, 2)]
ap2: [(0, 2)] [(0, 2, 'XX')]
[(0, 2, 'XX')]
reading miriad
ap1: [(0, 11)]
ap2: [(0, 11)] [(0, 11, 'XX')]
[(0, 11, 'XX')]
reading miriad
ap1: [(0, 12)]
ap2: [(0, 12)] [(0, 12, 'XX')]
[(0, 12, 'XX')]
reading miriad
ap1: [(0, 13)]
ap2: [(0, 13)] [(0, 13, 'XX')]
[(0, 13, 'XX')]
reading miriad
ap1: [(0, 14)]
ap2: [(0, 14)] [(0, 14, 'XX')]
[(0, 14, 'XX')]
reading miriad
ap1: [(0, 23)]
ap2: [(0, 23)] [(0, 23, 'XX')]
[(0, 23, 'XX')]
reading miriad
ap1: [(0, 24)]
ap2: [(0, 24)] [(0, 24, 'XX')]
[(0, 24, 'XX')]
reading miriad
ap1: [(0, 25)]
ap2: [(0, 25)] [(0, 25, 'XX')]
[(0, 25, 'XX')]
reading miriad
ap1: [(0, 26)]
ap2: [(0, 26)] [(0, 26, 'XX')]
[(0, 26, 'XX')]
reading miriad
ap1: [(0, 27)]
ap2: [(0, 27)] [(0, 27, 'XX')]
[(0, 27, 'XX')]
reading miriad
ap1: [(0, 36)]
ap2: [(0, 36)] [(0, 36, 'XX')]
[(0, 36, 'XX')]
reading mir

KeyboardInterrupt: 

In [98]:
d,f,s = hd.read_data(bls=[(0,1,'xx')])

self.antpairs [(0, 1)]


In [91]:
hd._blt_slices

{(0, 1): slice(0, 60, 1)}

In [22]:
d.keys()

[(0, 1, 'XX')]

In [6]:
uvd = UVData('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/'')
uvd.read_miriad_metadata(/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/')

uvd2 = UVData()
uvd2.read_miriad_metadata('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uvO/')

In [82]:
uvd._Nfreqs.required

True

In [20]:
uvd2._antenna_names

In [12]:
for key in uvd2.__dict__.keys():
    print key, eval('uvd2.' + key)

 _uvplane_reference_time <pyuvdata.parameter.UVParameter object at 0x111941710>
_Nspws <pyuvdata.parameter.UVParameter object at 0x1118f7250>
_antenna_numbers <pyuvdata.parameter.UVParameter object at 0x111941690>
_vis_units <pyuvdata.parameter.UVParameter object at 0x1118f7850>
_antenna_names <pyuvdata.parameter.UVParameter object at 0x111941c10>
_phase_type <pyuvdata.parameter.UVParameter object at 0x111941c50>
_nsample_array <pyuvdata.parameter.UVParameter object at 0x1118f71d0>
_data_array <pyuvdata.parameter.UVParameter object at 0x1118f7810>
_spw_array <pyuvdata.parameter.UVParameter object at 0x1118f77d0>
_Nfreqs <pyuvdata.parameter.UVParameter object at 0x1118f7450>
_gst0 <pyuvdata.parameter.UVParameter object at 0x111941fd0>
_earth_omega <pyuvdata.parameter.UVParameter object at 0x111941d90>
_phase_center_ra <pyuvdata.parameter.AngleParameter object at 0x111941d10>
_Ntimes <pyuvdata.parameter.UVParameter object at 0x1118f7550>
_Npols <pyuvdata.parameter.UVParameter object at 0

In [8]:
for key, val in uvd2.__dict__.items():
    
    if not (uvd.__dict__[key] == val):
        print key
        print uvd.__dict__[key] == val

history parameter value is a string, values are different
_history
history parameter value is a string, values are different
False


In [7]:
uvd.__add__(uvd2, run_check=False, check_extra=False, run_check_acceptability=False)

TypeError: zip argument #1 must support iteration

In [62]:
uvd = UVData()
print uvd.read_miriad_metadata('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/')

None


In [64]:
#hd = HERAData(uvd)
hd = HERAData('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/')
print hd.ants

[136 140 121  88  41  86  36  51  50  98 123 124  65 137  82 120 143  66
  83 122  67  68  69  70  71  85   0   1  11  12  13  14   2  23  24  25
  26  27  37  38  39  40  52  53  54  55  84  87 141 138 142 139]


In [65]:
hd.polarization_array

In [68]:
hd = HERAData(['/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/', '/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv/'])
# uvd.antenna_diameters /= 2
# print uvd.antenna_diameters
# print hd.antenna_diameters

ValueError: A list of files cannot be used when just reading metadata

In [31]:
hd.check()

True

In [39]:
# class HERACal(UVCal):
    


class HERAData(UVData):
    
    
    '''MY IDEA: MAKE A HERA DATA OBJECT INSTANTIATED WITH A FILE NAME THAT LOADS IN META DATA AND PUTS IT IN A NICE FORM
    THIS THEN PROVIDES AN INTERFACE TO PARTIAL DATA LOADING AND ALSO PARTIAL DATA WRITING WHICH SPITS OUT DATA CONTAINERS WHICH HAVE COPIES
    OF THE DATA. '''

    def __init__(self, input_data, filetype='miriad'):
        
        if isinstance(input_data, UVData):
            pass 
        else:
            super(HERAData, self).__init__()

    
    def read_data(self):
        print(testing)
        
        
#     def _build_dc(self, data_array):
        
#         dc = 
        
#         self.freqs = np.unique(uvd.freq_array)
#         self.times = np.unique(uvd.time_array)
#         lst_indices = np.unique(uvd.lst_array.ravel(), return_index=True)[1]
#         self.lsts = uvd.lst_array.ravel()[np.sort(lst_indices)]
#         antpos, self.ants = uvd.get_ENU_antpos(center=True, pick_data_ants=True)
#         self.antpos = dict(zip(self.ants, antpos))
#         self.pols = np.array([polnum2str(polnum) for polnum in uvd.polarization_array])
    
#     def _determine_slicing(self)
#         # Figure out baseline slicing
#         self._blt_slices = {}
#         self.antpairs = uvd.get_antpairs()
#         for ant1, ant2 in self.antpairs:
#             indices = uvd.antpair2ind(ant1, ant2)
#             if not (len(set(np.ediff1d(indices))) == 1):
#                 raise NotImplementedError('UVData objects with non-regular spacing of baselines in its baseline-times are not supported.')
#             self._blt_slices[(ant1, ant2)] = slice(indices[0], indices[-1] +1 , indices[1] - indices[0])
        
#         # Figure out polarization indexing
#         self._polnum_indices = {}
#         for i, polnum in enumerate(uvd.polarization_array):
#             self._polnum_indices[polnum] = i
    
#     def apply_cal()
    
#     def get_dcs(self):
#         '''TODO: document (makes a copy of the data)'''
#         if self.data_array is None:
#             raise ValueError('Cannot build DataContainers if no data has been loading into this object.')
            
#         self._blt_slices = {}
#         self.antpairs = uvd.get_antpairs()
#         for ant1, ant2 in self.antpairs:
#             indices = uvd.antpair2ind(ant1, ant2)
#             if not (len(set(np.ediff1d(indices))) == 1):
#                 raise NotImplementedError('UVData objects with non-regular spacing of baselines in its baseline-times are not supported.')
#             self._blt_slices[(ant1, ant2)] = slice(indices[0], indices[-1] +1 , indices[1] - indices[0])     
            
#         data = self._build_dc(self.data_array)
#         flags = self._build_dc(self.flag_array)
#         nsamples = self._build_dc(self.nsample_array)
        
        
        
    
#     def set_dcs(self, )

IndentationError: expected an indented block (<ipython-input-39-ce936cfab28f>, line 19)

In [ ]:
uvdc.nsample_array

In [28]:
def test(a=2):
    print *args

SyntaxError: invalid syntax (<ipython-input-28-c67386ee7bab>, line 2)

# meeting with aaron

* make vis_data a subclass of UVData
* also make cal_data a subclass UVCal
* make vis_data generate a set of data containers
* make data containers store useful metadata
* make data containers instantiated from dictionaries not arrays
* make data containers calibratable using cal_container

CONCLUSION: 
* Instantiate data_containers upon read hd.data, hd.flags, hd.nsamples which are data containers with read-only views of the data
* Allow for a hd.get_dc (which always produces copies by default) and hd.set_dc which slurps in data containers



In [22]:

    
class CalDict:
    '''TODO: document'''

    def __init__(self, cal_array, ant_indices, jnum_indices):
        '''TODO: document'''
        self._cal_array = cal_array
        self._ant_indices = ant_indices
        self._jnum_indices = jnum_indices
        
        self.pols = sorted([jnum2str(jnum) for jnum in self._jnum_indices.keys()])
        self.ants = sorted(ant_indices.keys())
        self.dict = {(ant,pol): self[ant,pol] for ant in self.ants for pol in self.pols}
    
    def __getitem__(self, key):
        '''TODO: document'''
        pass

    def __setitem__(self, key, value):
        '''TODO: document'''
        pass
    
    def keys(self):
        '''TODO: document'''
        return self.dict.keys()

    def values(self):
        '''TODO: document'''        
        return self.dict.values()
    
    def items(self):
        '''TODO: document'''
        return self.dict.items()
    
    def __len__(self):
        '''TODO: document'''        
        return len(self.dict)
    
    def __contains__(self, key):
        '''TODO: document'''
        try:
            self[key]
            return True
        except KeyError:
            return False    
    
class DataDict:
    '''TODO: document'''
    
    def __init__(self, data_array, blt_slices, polnum_indices):
        '''TODO: document'''
        self._data_array = data_array
        self._blt_slices = blt_slices
        self._polnum_indices = polnum_indices
        self._gains = None
        self._cal_flags = None
        self._flags = None
        
        self.pols = sorted([polnum2str(polnum) for polnum in self._polnum_indices.keys()])
        self.antpairs = sorted(blt_slices.keys())
        self.dict = {antpair + (pol,): self[antpair + (pol,)] 
                     for antpair in self.antpairs for pol in self.pols}
    
    def apply_calibration(self, gain_dict):
        '''TODO: document'''
        assert(np.iscomplexobj(self._data_array))
        self._gains = {(ant, jstr2num(ap)): gain_dict[ant, ap] for (ant, ap) in gain_dict.keys()}

    def apply_cal_flags(self, cal_flags_dict):
        '''TODO: document'''
        assert(self._data_array.dtype is bool)
        self._cal_flags = {(ant, jstr2num(ap)): gain_dict[ant, ap] for (ant, ap) in gain_dict.keys()}
        
    def apply_vis_flags(self, vis_flags_dict):
        '''TODO: document'''
        assert(self._data_array.dtype is bool)
        self._flags = {(ant1, ant2, polstr2num(pol)): vis_flags_dict[ant1, ant2, pol] 
                                                      for (ant1, ant2, pol) in vis_flags_dict.keys()}
    
    def __getitem__(self, key):
        '''TODO: REWRITE... Returns the data corresponding to the key. If the key is just a polarization,
        returns all baselines for that polarization. If the key is just a baseline,
        returns all polarizations for that baseline. If the key is of the form (0,1,'xx'),
        returns the associated entry. Abstracts away both baseline ordering (applying the
        complex conjugate when appropriate) and polarization capitalization.'''
        if isinstance(key, tuple) and len(key) == 3:  # asking for bl-pol
            try:
                out = np.conj(np.squeeze(self._data_array[self._blt_slices[key[1::-1]], 
                                         0, : , self._polnum_indices[polstr2num(conj_pol(key[2]))]]))
            except:
                out = np.squeeze(self._data_array[self._blt_slices[key[0:2]], 
                                 0, : , self._polnum_indices[polstr2num(key[2])]])
            
            if self._gains is not None:  # divide out calibration solutions
                jn_i, jn_j = jstr2num([jstr for jstr in split_pol(key[2])])
                out = out / self._gains[key[0], jn_i] / np.conj(self._gains[key[1], jn_j])
            
            if self._cal_flags is not None:  # incorporate per-antenna flags
                jn_i, jn_j = split_pol(key[2])
                out = out | self._cal_flags[key[0], jstr2num(jn_i)] | self._cal_flags[key[1], jstr2num(jn_j)]

            if self._cal_flags is not None:  # incorporate visibility flags
                ap_i, ap_j = split_pol()
                out = out | self._cal_flags[key[0], jstr2num(ap_i)] | self._cal_flags[key[1], jstr2num(ap_j)]
                
            out.flags.writeable = False  # creates read-only views
            return out
        elif isinstance(key, tuple) and len(key) == 2:  # asking for antpair
            return {pol: self[key + (pol,)] for pol in self.pols}
        elif isinstance(key, str):  # asking for a pol
            return {antpair: self[antpair + (key,)] for antpair in self.antpairs}
        else:
            raise KeyError('Unrecognized key type for slicing data.')

    def __setitem__(self, key, value):
        '''TODO: REWRITE... Sets the data corresponding to the key. Only supports the form (0,1,'xx').
        Abstracts away both baseline ordering (applying the complex conjugate when
        appropriate) and polarization capitalization.'''
        if isinstance(key, tuple) and len(key) == 3:  # providing bl-pol
            try:
                self._data_array[self._blt_slices[key[0:2]], 0, : , 
                                 self._polnum_indices[polstr2num(key[2])]] = value
            except(KeyError):
                self._data_array[self._blt_slices[key[1::-1]], 0, : , 
                                 self._polnum_indices[polstr2num(conj_pol(key[2]))]] = np.conj(value)
        elif isinstance(key, tuple) and len(key) == 2:  # providing antpair with all pols
            for pol in value.keys():
                self[key + (pol,)] = value[pol]
        elif isinstance(key, str):  # providing pol with all antpairs
            for antpair in value.keys():
                self[antpair + (key,)] = value[key]
        else:
            raise KeyError('Unrecognized key type for slicing data.')

    def keys(self):
        '''TODO: document'''
        return self.dict.keys()

    def values(self):
        '''TODO: document'''        
        return self.dict.values()
    
    def items(self):
        '''TODO: document'''
        return self.dict.items()
    
    def __len__(self):
        '''TODO: document'''        
        return len(self.dict)
    
    def __contains__(self, key):
        '''TODO: document'''
        try:
            self[key]
            return True
        except KeyError:
            return False

    def get_data(self, *args):
        '''TODO: document'''
        if len(args) > 1:
            return self[tuple(args)]
        else:
            return self.__getitem__(*args)
            
class VisData:
    '''TODO: document'''

    def __init__(self, uvd):
        '''TODO: document
        Arguments:
            uvd: pyuvdata object
        '''
        
        self.uvd = uvd
        if uvd.data_array.shape[1] != 1: 
            raise NotImplementedError('Multiple spectral windows are not supported by VisData.')
        
        # Figure out baseline slicing
        self._blt_slices = {}
        self.antpairs = uvd.get_antpairs()
        for ant1, ant2 in self.antpairs:
            indices = uvd.antpair2ind(ant1, ant2)
            if not (len(set(np.ediff1d(indices))) == 1):
                raise NotImplementedError('UVData objects with non-regular spacing of baselines in its baseline-times are not supported.')
            self._blt_slices[(ant1, ant2)] = slice(indices[0], indices[-1] +1 , indices[1] - indices[0])
        
        # Figure out polarization indexing
        self._polnum_indices = {}
        for i, polnum in enumerate(uvd.polarization_array):
            self._polnum_indices[polnum] = i
            
        # Build DataDicts of data, flags, and nsamples
        self.data = DataDict(uvd.data_array, self._blt_slices, self._polnum_indices)
        self.flags = DataDict(uvd.flag_array, self._blt_slices, self._polnum_indices)
        self.nsamples = DataDict(uvd.nsample_array, self._blt_slices, self._polnum_indices)
        self._primary_array = self.data
        
        # Extract useful metadata
        self.freqs = np.unique(uvd.freq_array)
        self.times = np.unique(uvd.time_array)
        lst_indices = np.unique(uvd.lst_array.ravel(), return_index=True)[1]
        self.lsts = uvd.lst_array.ravel()[np.sort(lst_indices)]
        antpos, self.ants = uvd.get_ENU_antpos(center=True, pick_data_ants=True)
        self.antpos = dict(zip(self.ants, antpos))
        self.pols = np.array([polnum2str(polnum) for polnum in uvd.polarization_array])
        self.integration_time = self.uvd.integration_time
    

    def get_lsts(self, bl):
        '''TODO: Document'''
        pass
    
    def get_lsts(self, bl):
        '''TODO: Document'''
        pass
    
    def apply_calibration(self, gains=None, cal_flags=None):
        '''TODO: Document'''
        if gains is not None:
            self.data.apply_calibration(gains)
        if cal_flags is not None:
            self.flags.apply_cal_flags(cal_flags)
    
    def apply_flags(self, flags=None):
        '''TODO: Document'''
        if flags is not None:
            self.flags.apply_vis_flags(flags)
        pass
    
    def __getitem__(self, key):
        '''TODO: Document'''
        return self._primary_array[key]

    def __setitem__(self, key, value):
        '''TODO: Document'''
        self._primary_array[key] = value

    def keys(self):
        '''TODO: document'''
        return self._primary_array.keys()

    def values(self):
        '''TODO: document'''        
        return self._primary_array.values()
    
    def items(self):
        '''TODO: document'''
        return self._primary_array.items()
    
    def __len__(self):
        '''TODO: document'''        
        return len(self._primary_array.dict)
    
    def __contains__(self, key):
        '''TODO: document'''
        try:
            self._primary_array[key]
            return True
        except KeyError:
            return False

    def get_data(self, *args):
        '''TODO: document'''
        if len(args) > 1:
            return self._primary_array[tuple(args)]
        else:
            return self._primary_array.__getitem__(*args)

In [23]:
data = VisData(uvd)

In [24]:
io.write_vis()

TypeError: write_vis() takes at least 5 arguments (0 given)

In [25]:
data._polnum_indices

{-5: 0}

In [26]:
data.nsamples[0,1,'xx'] == data.nsamples[1,0,'XX']

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ..., 
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [28]:
gains, gain_flags = io.load_cal('/Users/jsdillon/Desktop/zen.2458101.65493.xx.HH.uv.abs.calfits')

# proto unit tests

In [29]:
data = VisData(uvd)
test = data[53,54,'xx']
print np.all(test == data.uvd.get_data(53,54,'xx'))
test_conj = data[54,53,'xx']
print np.all(test_conj == data.uvd.get_data(54,53,'xx'))
data.apply_calibration(gains)
print np.sum(test / gains[53,'x'] / np.conj(gains[54,'x']) - data[53,54,'xx'])
print np.sum(test_conj / gains[54,'x'] / np.conj(gains[53,'x']) - data[54,53,'xx'])

True
True
0j
0j


In [30]:
for key in data.flags.keys():
    data.flags[key] = np.ones_like(data.flags[key])

In [34]:
np.all(data.uvd.flag_array)

True

In [351]:
np.all(np.conj(data.data['xx'][0,1]) == uvd.get_data(1,0))

True

In [152]:
uvd.get_data(0,1).shape

(60, 1024)

In [139]:
uvd.polarization_array

array([-5])

In [299]:
np.iscomplexobj(data.data._data_array)

True

In [302]:
data.flags._data_array.dtype == bool

True

In [143]:
data.data[0,1,'xx'] *= 2

ValueError: output array is read-only

In [317]:
True | False | False


True

In [ ]:
data .

In [38]:
test = np.zeros((5,5))
blah = {'a': test[:,0]}
blah['a'] = np.ones(5)
print test

[[ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.]]
